# INFO 212: Data Science Programming 1
## CCI at Drexel University


## Week 9: Lab: Data Loading and Merging


In [1]:
# Importing neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Exercise 1

1: Read in the tables from the Wikipedia page about World Series champions: https://en.wikipedia.org/wiki/List_of_World_Series_champions

In [2]:
# Set the target URL to the Wikipedia page of World Series champions
target_url = 'https://en.wikipedia.org/wiki/List_of_World_Series_champions'

# Send a GET request to fetch the HTML content of the page
html_source = requests.get(target_url)

# Check the status code to ensure the request was successful
html_source.status_code

# Access the text content of the response
html_source.text

# Use pandas to read all tables from the HTML content
tables = pd.read_html(html_source.text)

/var/folders/mw/4snkpx0970n_6rz3jtvq4yxr0000gn/T/ipykernel_15553/3801525153.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_source.text)


2: How many tables have you read in? List the columns of the table containing the world series results.

In [3]:
# Print the number of tables extracted from the HTML content
print(f"Number of tables read: {len(tables)}\n")

# Loop through each table along with its index
for i, table in enumerate(tables):
    # print(f"Table {i+1} Columns: {table.columns.tolist()}\n")
    # Check if the current table contains the 'Series' column
    if 'Series' in table.columns:
        # Assign this table to 'world_series_table'
        world_series_table = table
        # Exit the loop
        break

# Display the columns of the World Series table
print(f"The columns of the World Series Table are: {world_series_table.columns}")


Number of tables read: 27

The columns of the World Series Table are: Index(['Year', 'Winning team', 'Manager', 'Series', 'Losing team', 'Manager.1',
       'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')


3: List all the world series records that a team from Philadelphia played.  

In [4]:
# Loop through each table along with its index
for i, table in enumerate(tables):
    # print(f"Table {i+1} Columns: {table.columns.tolist()}\n")
    # Check if the current table contains the 'Win%' column
    if 'Win\xa0%' in table.columns:
        # Assign this table to 'world_series_records_by_franchise'
        world_series_records_by_franchise = table
        # Exit the loop
        break

# Filter the DataFrame for teams containing "Philadelphia" in their name
philly_teams = world_series_records_by_franchise[world_series_records_by_franchise['Team'].str.contains("Philadelphia")]

# Print the filtered DataFrame of Philadelphia teams
print(philly_teams)

                                              Team  Wins  Losses  Apps Win %  \
2   Philadelphia / Kansas City / Oakland Athletics     9       5    14  .643   
14                           Philadelphia Phillies     2       6     8  .250   

                                            Season(s)  
2   1905, 1910, 1911, 1913, 1914, 1929, 1930, 1931...  
14     1915, 1950, 1980, 1983, 1993, 2008, 2009, 2022  


4: What is the winning probability of the Phillies?

In [5]:
# Loop through each table along with its index
for i, table in enumerate(tables):
    # print(f"Table {i+1} Columns: {table.columns.tolist()}\n")
    # Check if the current table contains the 'Win%' column
    if 'Win\xa0%' in table.columns:
        # Assign this table to 'world_series_records_by_franchise' 
        world_series_records_by_franchise = table
        # Exit the loop
        break

# Retrieve the 'Win%' value for the 'Philadelphia Phillies' from the DataFrame
phillies_win_percentage = world_series_records_by_franchise.loc[world_series_records_by_franchise['Team'] == 'Philadelphia Phillies', 'Win\xa0%']

# Calculate and print the winning probability formatted to two decimal places
print(f"The winning probability of the Phillies is: {(float(phillies_win_percentage.iloc[0]) * 100):.2f}%")

The winning probability of the Phillies is: 25.00%


# Exercise 2

1: Read in the roam_prescription_based_prediction.jsonl using Pandas. This is the dataset used in the Roam blog post Prescription-based prediction. The file is in JSONL format (one JSON record per line). Show the size of the data, columns, etc.

In [6]:
# Loading the jsonl file
df = pd.read_json("/Users/mustafabookwala/Desktop/Drexel/Pre-Junior/Fall 2024/INFO 212/Week 9/roam_prescription_based_prediction.jsonl", lines=True)

# Displaying relevant information
print(f"Number of Rows of the dataframe: {df.shape[0]}\n")
print(f"Number of Columns of the dataframe: {df.shape[1]}\n")
print(f"Columns of the dataframe: {df.columns}\n")
print(f"Displaying the first 5 values of the dataframe: {df.head()}\n")
print(f"{df.describe()}")

Number of Rows of the dataframe: 239930

Number of Columns of the dataframe: 3

Columns of the dataframe: Index(['cms_prescription_counts', 'provider_variables', 'npi'], dtype='object')

Displaying the first 5 values of the dataframe:                              cms_prescription_counts  \
0  {'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...   
1  {'CEPHALEXIN': 23, 'AMOXICILLIN': 52, 'HYDROCO...   
2  {'CEPHALEXIN': 28, 'AMOXICILLIN': 73, 'CLINDAM...   
3                                {'AMOXICILLIN': 63}   
4  {'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...   

                                  provider_variables         npi  
0  {'settlement_type': 'non-urban', 'generic_rx_c...  1295763035  
1  {'settlement_type': 'non-urban', 'generic_rx_c...  1992715205  
2  {'settlement_type': 'non-urban', 'generic_rx_c...  1578587630  
3  {'settlement_type': 'non-urban', 'generic_rx_c...  1932278405  
4  {'settlement_type': 'non-urban', 'generic_rx_c...  1437366804  

                npi
count

2: Extract the first 10,000 provider information as a DataFrame called `providers`. Set the index to be the npi.

In [7]:
# Extract the first 10,000 rows for 'provider_variables' and 'npi' columns from the DataFrame
providers = df.loc[:9999, ["provider_variables", "npi"]]

# Set 'npi' column as the index of the DataFrame
providers.set_index('npi', inplace=True)

# Display the resulting DataFrame
print(providers)

                                           provider_variables
npi                                                          
1295763035  {'settlement_type': 'non-urban', 'generic_rx_c...
1992715205  {'settlement_type': 'non-urban', 'generic_rx_c...
1578587630  {'settlement_type': 'non-urban', 'generic_rx_c...
1932278405  {'settlement_type': 'non-urban', 'generic_rx_c...
1437366804  {'settlement_type': 'non-urban', 'generic_rx_c...
...                                                       ...
1700873247  {'settlement_type': 'non-urban', 'generic_rx_c...
1689729568  {'settlement_type': 'urban', 'generic_rx_count...
1649455882  {'settlement_type': 'non-urban', 'generic_rx_c...
1396938197  {'settlement_type': 'non-urban', 'generic_rx_c...
1588704415  {'settlement_type': 'urban', 'generic_rx_count...

[10000 rows x 1 columns]


3: Extract first 10,000 prescription counts as a DataFrame called `pres`. Set the index to be the npi.

In [8]:
# Extract the first 10,000 rows for 'cms_prescription_counts' and 'npi' columns from the DataFrame
pres = df.loc[:9999, ["cms_prescription_counts", "npi"]]

# Set 'npi' column as the index of the DataFrame
pres.set_index('npi', inplace=True)

# Display the resulting DataFrame
print(pres)

                                      cms_prescription_counts
npi                                                          
1295763035  {'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...
1992715205  {'CEPHALEXIN': 23, 'AMOXICILLIN': 52, 'HYDROCO...
1578587630  {'CEPHALEXIN': 28, 'AMOXICILLIN': 73, 'CLINDAM...
1932278405                                {'AMOXICILLIN': 63}
1437366804  {'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...
...                                                       ...
1700873247  {'DILTIAZEM 24HR ER': 49, 'ENALAPRIL MALEATE':...
1689729568                                      {'CREON': 14}
1649455882                                  {'IBUPROFEN': 24}
1396938197  {'FLUOXETINE HCL': 19, 'SIMVASTATIN': 32, 'NOV...
1588704415  {'SERTRALINE HCL': 22, 'TRAZODONE HCL': 17, 'S...

[10000 rows x 1 columns]


4: Join `providers` with `pres` on npi. Extract the providers' info

*   List item
*   List item

who prescribed the most number of DIAZEPAM.

In [9]:
# Concatenate the 'providers' and 'pres' DataFrames along the columns
providers_and_pres = pd.concat([providers, pres], axis=1)

# Ensure the index is of type string for consistency
providers_and_pres.index = providers_and_pres.index.astype(str)

# Initialize a list to store diazepam prescription counts for each provider
diazepam_counts = []

# Iterate over each provider in the DataFrame
for index in providers_and_pres.index:
    # Get the CMS prescription counts for the current provider
    cms_counts = providers_and_pres.loc[index, 'cms_prescription_counts']
    # Check if 'DIAZEPAM' is among the prescriptions
    if 'DIAZEPAM' in cms_counts:
        # Get the count of 'DIAZEPAM' prescriptions
        count = cms_counts['DIAZEPAM']
    else:
        # If not prescribed, set count to zero
        count = 0
    # Append the provider's NPI and diazepam count to the list
    diazepam_counts.append((index, count))

# Create a DataFrame from the list of diazepam counts
diazepam = pd.DataFrame(diazepam_counts, columns=['npi', 'DIAZEPAM_count'])

# Set 'npi' as the index of the new DataFrame
diazepam.set_index('npi', inplace=True)

# Index must be of type string
diazepam.index = diazepam.index.astype(str)

# Merge the diazepam counts back into the main DataFrame
providers_and_pres = providers_and_pres.merge(diazepam, left_index=True, right_index=True)

# Find the providers with the maximum diazepam prescription count
max_diazepam_providers = providers_and_pres[
    providers_and_pres['DIAZEPAM_count'] == providers_and_pres['DIAZEPAM_count'].max()
]

# Iterate over the providers with the maximum diazepam prescriptions
for index in max_diazepam_providers.index:
    # Get the provider's information
    provider_info = max_diazepam_providers.loc[index, 'provider_variables']
    # Print the provider's NPI
    print(f"provider_npi: {index}")
    # Print the maximum diazepam prescription count
    print(f"DIAZEPAM count: {max_diazepam_providers['DIAZEPAM_count'].max()}")
    # Print each detail about the provider
    for key, value in provider_info.items():
        print(f"{key}: {value}")

provider_npi: 1730150194
DIAZEPAM count: 1327
settlement_type: non-urban
generic_rx_count: 23842
specialty: Interventional Pain Medicine
years_practicing: 7
gender: M
region: South
brand_name_rx_count: 3264
